# Импортирање на потребните библиотеки

In [ ]:
from keras.layers import Dense
from keras.layers import Input
from keras.models import Sequential, Model
from sklearn.datasets import load_iris
from sklearn.preprocessing import LabelBinarizer
from tensorflow.distribute import MirroredStrategy, MultiWorkerMirroredStrategy

# Тренирање и евалуација на модели

## Вчитување податоци

In [ ]:
# го вчитуваме податочното множество Iris кое е составено од информации за
# карактеристики за цвеќиња заедно со нивниот тип кој претставува класа
# со поставување на атрибутот return_X_y на вредност True го вчитуваме
# множеството во 2 низи (1 за карактеристиките и 1 за класите)
features, labels = load_iris(return_X_y=True)

# за класификација потребно е класите да ги претставиме како one-hot вектори
# за таа цел инстанцираме LabelBinarizer објект
label_binarizer = LabelBinarizer()
label_binarizer.fit_transform([0, 1, 2])

# ги трансформираме класите во one-hot вектори
labels = label_binarizer.transform(labels)

# го делиме множеството на подмножества за тренирање и тестирање
# 80% од множеството за тренирање и 20% од множеството за тестирање
train_x, train_y = features[:int(0.8 * len(features))], labels[:int(0.8 * len(labels))]
test_x, test_y = features[int(0.8 * len(features)):], labels[int(0.8 * len(labels)):]

## Креирање Sequential модел

In [ ]:
# за почеток се инстанцира објект кој ќе го претставува моделот
model1 = Sequential() 

# потоа може да се додаваат неограничен број слоеви во зависност од потребите
# за конкретниот проблем кој се разгледува

# во продолжение додаваме 3 целосно поврзани слоеви со 64, 128 и 512 
# неврони соодветно, и активациска функција ReLu
# кај првиот слој потребно е да се дефинира атрибутот input_shape кој ја 
# претставува димензијата на влезните податоци
# (4, ) означува дека моделот ќе добива 4-димензионални вектори на влез
model1.add(Dense(64, activation='relu', name='l1', input_shape=(4,)))
model1.add(Dense(128, activation='relu', name='l2')) 
model1.add(Dense(512, activation='relu')) 

# последниот слој претставува излезен слој кој има 3 неврони и активациска
# функција softmax што означува дека моделот ќе предвидува реални броеви кои
# може да се интерпретираат како веројатности за припадност во класа
model1.add(Dense(3, activation='softmax'))

## Креирање Model модел

In [ ]:
# за креирање модел преку функционалното API потребно е да се дефинира 
# влезен слој со димензија која е соодветна на димензијата на влезните податоци
a = Input(shape=(4,)) 

# потоа може да се додаваат неограничен број слоеви во зависност од потребите
# за конкретниот проблем кој се разгледува

# во продолжение додаваме 3 целосно поврзани слоеви со 64, 128 и 256 
# неврони соодветно, и активациска функција ReLu
x = Dense(64, activation='relu', name='l1')(a) 
x = Dense(128, activation='relu', name='l2')(x) 
x = Dense(256, activation='relu')(x) 

# последниот слој претставува излезен слој кој има 3 неврони и активациска
# функција softmax што означува дека моделот ќе предвидува реални броеви кои
# може да се интерпретираат како веројатности за припадност во класа
b = Dense(3, activation='softmax')(x) 

# по дефинирање и поврзување на слоевите, се креира објект кој ќе го
# претставува моделот при што се поставуваат параметрите inputs и outputs
# кои претставуваат влезни и излезни слоеви на моделот, соодветно
model2 = Model(inputs=a, outputs=b)

## Тренирање

In [ ]:
# пред да се тренира, потребно е моделот да се компајлира
# со поставување на аргументот loss дефинираме категориска крос-ентропија
# како функција на загуба
# со поставување на аргументот optimizer дефинираме Adam оптимизатор
# со поставување на аргументот metrics дефинираме точност како метрика за 
# следење на перформансите на моделот при тренирање
model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# при тренирање на моделот покрај влезните и излезните податоци, потребно е 
# да се постават вредности за аргументите batch_size што претставува големина
# на серија, epochs што претставува број на епохи и опционално verbose со што 
# се дефинира интензитет на информации за процесот на тренирање (0 - никакви
# информации, 1 - информации за секој чекор, 
# 2 - сумирани информации по секоја епоха)
model1.fit(train_x, train_y, batch_size=32, epochs=10, verbose=2)

Epoch 1/10
4/4 - 1s - loss: 1.0273 - accuracy: 0.4750
Epoch 2/10
4/4 - 0s - loss: 0.8276 - accuracy: 0.8167
Epoch 3/10
4/4 - 0s - loss: 0.6549 - accuracy: 0.8333
Epoch 4/10
4/4 - 0s - loss: 0.5311 - accuracy: 0.8333
Epoch 5/10
4/4 - 0s - loss: 0.4319 - accuracy: 0.8333
Epoch 6/10
4/4 - 0s - loss: 0.3720 - accuracy: 0.8333
Epoch 7/10
4/4 - 0s - loss: 0.3302 - accuracy: 0.8333
Epoch 8/10
4/4 - 0s - loss: 0.3001 - accuracy: 0.8833
Epoch 9/10
4/4 - 0s - loss: 0.2656 - accuracy: 0.9083
Epoch 10/10
4/4 - 0s - loss: 0.2360 - accuracy: 0.9000


## Евалуација

In [ ]:
# евалуација на моделот со што се добиваат вредности за
# функцијата на загуба и точноста
model1.evaluate(test_x, test_y)

1/1 [==============================] - 0s 172ms/step - loss: 0.5985 - accuracy: 0.7000


[0.5985440015792847, 0.699999988079071]

## Тренирање на повеќе GPU (single host)

In [ ]:
# за тренирање на повеќе графички картички на еден уред треба да се инстанцира
# објект со соодветната стратегија - MirroredStrategy 
strategy = MirroredStrategy()

# потоа во scope на дефинираната стратегија се креира и компајлира моделот
# на овој начин се прави копија на моделот на сите графички картички така што
# секоја копија ќе користи различен дел од податоците
with strategy.scope():
    model3 = Sequential()
    model3.add(Dense(64, activation='relu', name='l1', input_shape=(4,)))
    model3.add(Dense(128, activation='relu', name='l2')) 
    model3.add(Dense(512, activation='relu')) 
    model3.add(Dense(3, activation='softmax'))
    model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# начинот на тренирање и евалуација е идентичен
model3.fit(train_x, train_y, batch_size=32, epochs=50, verbose=2)
model3.evaluate(test_x, test_y)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Epoch 1/50
4/4 - 2s - loss: 1.1426 - accuracy: 0.3000
Epoch 2/50
4/4 - 0s - loss: 0.9239 - accuracy: 0.7000
Epoch 3/50
4/4 - 0s - loss: 0.7806 - accuracy: 0.8333
Epoch 4/50
4/4 - 0s - loss: 0.6266 - accuracy: 0.8333
Epoch 5/50
4/4 - 0s - loss: 0.5209 - accuracy: 0.8333
Epoch 6/50
4/4 - 0s - loss: 0.4275 - accuracy: 0.8333
Epoch 7/50
4/4 - 0s - loss: 0.3551 - accuracy: 0.8333
Epoch 8/50
4/4 - 0s - loss: 0.3105 - accuracy: 0.8917
Epoch 9/50
4/4 - 0s - loss: 0.2655 - accuracy: 0.9500
Epoch 10/50
4/4 - 0s - loss: 0.2472 - accuracy: 0.8500
Epoch 11/50
4/4 - 0s - loss: 0.2058 - accuracy: 0.9500
Epoch 12/50
4/4 - 0s - loss: 0.1816 - accuracy: 0.9750
Epoch 13/50
4/4 - 0s - loss: 0.1628 - accuracy: 0.9750
Epoch 14/50
4/4 - 0s - loss: 0.1376 - accuracy: 0.9667
Epoch 15/50
4/4 - 0s - loss: 0.1081 - accuracy: 0.9917
Epoch 16/50
4/4 - 0s - loss: 0.1169 - accuracy: 0.9750
Epoch 17/50
4/4 - 0s - loss

[0.5607110857963562, 0.7333333492279053]

## Тренирање на повеќе GPU (multiple workers)

In [ ]:
# за тренирање на повеќе графички картички на повеќе уреди треба да се
# инстанцира објект со соодветната стратегија - MultiWorkerMirroredStrategy 
strategy = MultiWorkerMirroredStrategy()

# потоа во scope на дефинираната стратегија се креира и компајлира моделот
# на овој начин се прави копија на моделот на сите графички картички на сите
# уреди така што секоја копија ќе користи различен дел од податоците
with strategy.scope():
    model4 = Sequential()
    model4.add(Dense(64, activation='relu', name='l1', input_shape=(4,)))
    model4.add(Dense(128, activation='relu', name='l2')) 
    model4.add(Dense(512, activation='relu')) 
    model4.add(Dense(3, activation='softmax'))
    model4.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# начинот на тренирање и евалуација е идентичен
model4.fit(train_x, train_y, batch_size=32, epochs=50, verbose=2)
model4.evaluate(test_x, test_y)

INFO:tensorflow:Single-worker MultiWorkerMirroredStrategy with local_devices = ('/device:CPU:0',), communication = CommunicationImplementation.AUTO
Epoch 1/50
4/4 - 3s - loss: 0.8882 - accuracy: 0.7000
Epoch 2/50
4/4 - 0s - loss: 0.6851 - accuracy: 0.8333
Epoch 3/50
4/4 - 0s - loss: 0.5085 - accuracy: 0.8333
Epoch 4/50
4/4 - 0s - loss: 0.4029 - accuracy: 0.8500
Epoch 5/50
4/4 - 0s - loss: 0.3253 - accuracy: 0.8417
Epoch 6/50
4/4 - 0s - loss: 0.2727 - accuracy: 0.8750
Epoch 7/50
4/4 - 0s - loss: 0.2383 - accuracy: 0.9833
Epoch 8/50
4/4 - 0s - loss: 0.1939 - accuracy: 0.9500
Epoch 9/50
4/4 - 0s - loss: 0.1753 - accuracy: 0.9500
Epoch 10/50
4/4 - 0s - loss: 0.1345 - accuracy: 0.9833
Epoch 11/50
4/4 - 0s - loss: 0.1232 - accuracy: 0.9583
Epoch 12/50
4/4 - 0s - loss: 0.1046 - accuracy: 0.9750
Epoch 13/50
4/4 - 0s - loss: 0.0826 - accuracy: 0.9833
Epoch 14/50
4/4 - 0s - loss: 0.0767 - accuracy: 0.9833
Epoch 15/50
4/4 - 0s - loss: 0.0731 - accuracy: 0.9833
Epoch 16/50
4/4 - 0s - loss: 0.0611 

[0.44764280319213867, 0.7333333492279053]

# Трансфер на знаење

In [ ]:
# зачувување на тежините на моделот како HDF5 датотека
model1.save_weights('model1_weights.h5')

In [ ]:
# при трансфер на знаење се вчитуваат тежините од некој претходно 
# истрениран модел така што моделот чии тежини се иницијализираат 
# треба да е однапред креиран
# со поставување на аргументот by_name на вредност True ќе се вчитаат
# тежините на сите слоеви според името на слојот
model2.load_weights('model1_weights.h5', by_name=True)

In [ ]:
# сумарни информации за моделот вклучувајќи информации за секој слој
# (име, димензија, број на тежини кои се учат)
# сите 42 435 тежини на моделот ќе се тренираат
model2.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
l1 (Dense)                   (None, 64)                320       
_________________________________________________________________
l2 (Dense)                   (None, 128)               8320      
_________________________________________________________________
dense_2 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 771       
Total params: 42,435
Trainable params: 42,435
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# со поставување на параметарот trainable на одреден слој
# означуваме дека тежините на тој слој нема да се тренираат
for layer in model2.layers[:2]:
    layer.trainable = False

In [ ]:
# сумарни информации за моделот вклучувајќи информации за секој слој
# (име, димензија, број на тежини кои се учат)
# ќе се тренираат 42 115 тежини од вкупно 42 435 тежини
model2.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
l1 (Dense)                   (None, 64)                320       
_________________________________________________________________
l2 (Dense)                   (None, 128)               8320      
_________________________________________________________________
dense_2 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 771       
Total params: 42,435
Trainable params: 42,115
Non-trainable params: 320
_________________________________________________________________
